# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent.

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev`
4. Install gymnasium with the needed extras

In [3]:
# %pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

Note: you may need to restart the kernel to use updated packages.


## The Lunar Lander Environment

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [2]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)

        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break

    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [5]:
import numpy as np

def random_agent_function(observation):
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [ 0.00645027  1.4181788   0.6533338   0.32258084 -0.00746751 -0.14798972
  0.          0.        ] -> Action: 2
Obs: [ 0.01283464  1.4262865   0.6461936   0.36028233 -0.0151318  -0.15329897
  0.          0.        ] -> Action: 2
Obs: [ 0.0190731   1.4344252   0.6323431   0.36161724 -0.02352365 -0.16785263
  0.          0.        ] -> Action: 3
Obs: [ 0.02539148  1.4419713   0.64233524  0.33518198 -0.03390992 -0.20774522
  0.          0.        ] -> Action: 0
Obs: [ 0.03171024  1.4489187   0.64236695  0.30850875 -0.04429396 -0.20769966
  0.          0.        ] -> Action: 3
Obs: [ 0.03809652  1.4552755   0.6508126   0.282121   -0.0563578  -0.24129912
  0.          0.        ] -> Action: 2
Obs: [ 0.04459019  1.4620398   0.6612022   0.30014405 -0.06808935 -0.23465283
  0.          0.        ] -> Action: 0
Obs: [ 0.05108423  1.4682056   0.6612351   0.2734569  -0.07981959 -0.23462597
  0.          0.        ] -> Action: 1
Obs: [ 0.05751104  1.4737837   0.65277517  0.24735028 -0.0898330

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

In [3]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [44]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:
        return Act.MAIN.value

    return Act.NO_OP.value

run_episode(rocket_agent_function)

Obs: [ 0.00353565  1.4165545   0.3581019   0.2504023  -0.00409007 -0.08111537
  0.          0.        ] -> Action: 0
Obs: [ 0.0070713   1.4216106   0.357618    0.22470444 -0.00809827 -0.08017045
  0.          0.        ] -> Action: 0
Obs: [ 0.01060705  1.4260676   0.35763055  0.19806772 -0.01210417 -0.08012564
  0.          0.        ] -> Action: 0
Obs: [ 0.01414309  1.4299248   0.3576424   0.17139561 -0.01610958 -0.08011521
  0.          0.        ] -> Action: 0
Obs: [ 0.01767912  1.4331822   0.35765427  0.14472578 -0.02011434 -0.08010263
  0.          0.        ] -> Action: 0
Obs: [ 0.02121534  1.43584     0.3576661   0.1180563  -0.02411847 -0.08008993
  0.          0.        ] -> Action: 0
Obs: [ 0.02475166  1.4378978   0.3576779   0.09138687 -0.02812198 -0.08007748
  0.          0.        ] -> Action: 0
Obs: [ 0.02828808  1.4393557   0.35768968  0.06471731 -0.03212485 -0.08006491
  0.          0.        ] -> Action: 0
Obs: [ 0.03182469  1.4402138   0.3577014   0.03804767 -0.0361271

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [7]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated:
            break

    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.

In [ ]:
def better_rocket_agent_function(observation):
    if observation[Obs.VY.value] < -0.2:
        return Act.MAIN.value
    
    if observation[Obs.ANGLE.value] > 0.1 or observation[Obs.ANGULAR_VELOCITY.value] > 2:
        return Act.RIGHT.value
    
    if observation[Obs.ANGLE.value] < -0.1 or observation[Obs.ANGULAR_VELOCITY.value] < -2:
        return Act.LEFT.value
    
    if observation[Obs.X.value] < -0.08:
        return Act.RIGHT.value
    if observation[Obs.X.value] > 0.08:
        return Act.LEFT.value
    
    if observation[Obs.LEFT_LEG_CONTACT.value] or observation[Obs.RIGHT_LEG_CONTACT.value]:
        if observation[Obs.ANGLE.value] > 0.4:
            return Act.RIGHT.value
        if observation[Obs.ANGLE.value] < -0.4:
            return Act.LEFT.value
        return Act.NO_OP.value
    
    return Act.NO_OP.value

# run_episode(better_rocket_agent_function, max_steps=1000)

rewards = run_episodes(better_rocket_agent_function)
print(rewards)
print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[100, -100, -100, 100, 100, 100, 100, 100, -100, 100, -100, 100, 100, np.float64(0.3089498373760773), -100, 100, 100, -100, 100, -100, 100, 100, 100, -100, -100, -100, -100, -100, 100, -100, -100, 100, -100, -100, -100, 100, -100, 100, 100, 100, 100, 100, 100, 100, -100, -100, 100, 100, -100, -100, -100, 100, -100, -100, -100, 100, 100, -100, 100, 100, 100, 100, 100, -100, 100, 100, 100, 100, 100, 100, -100, 100, 100, 100, 100, -100, -100, 100, 100, -100, 100, 100, 100, -100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, -100, -100, -100, -100, 100, -100, 100, 100, 100, -100, -100, 100, 100, 100, 100, 100, -100, -100, 100, -100, -100, 100, np.float64(-0.49122929381073877), 100, -100, 100, 100, -100, -100, 100, 100, 100, 100, 100, -100, -100, -100, 100, 100, -100, -100, -100, -100, 100, -100, 100, -100, 100, -100, 100, -100, 100, 100, 100, -100, 100, 100, 100, -100, -100, 100, 100, 100, -100, 100, -100, 100, -100, -100, 100, 100, -100, 100, 100, -100, 100, -100, 100, -100, -100, 100